In [2]:
import pandas as pd
from datetime import datetime

In [92]:
INVOICES_PATH = "../invoices.csv"
CREDIT_NOTES_PATH = "../credit_notes.csv"
PAYMENTS_PATH = "../payments.csv"

AS_AT_DATE = pd.Timestamp("2025-07-07")


BUCKETS = {
    "day_30": (0, 30),
    "day_60": (31, 60),
    "day_90": (61, 90),
    "day_120": (91, 120),
    "day_150": (121, 150),
    "day_180": (151, 180),
    "day_180_and_above": (181, float("inf")),
}   

In [32]:
def load_data():
    invoices = pd.read_csv(INVOICES_PATH, parse_dates=["invoice_date"])
    credit_notes = pd.read_csv(CREDIT_NOTES_PATH, parse_dates=["credit_note_date"])
    payments = pd.read_csv(PAYMENTS_PATH, parse_dates=["payment_date"])
    return invoices, credit_notes, payments

In [33]:
invoices, credit_notes, payments = load_data()

In [34]:
invoices.head()

,id,centre_id,class_id,student_id,invoice_date,total_amount
0,inv_001,c_01,cls_01,stu_001,2025-05-01,300.0
1,inv_002,c_01,cls_02,stu_002,2025-06-01,200.0
2,inv_003,c_02,cls_01,stu_003,2025-01-01,500.0
3,inv_004,c_03,cls_03,stu_004,2024-12-15,400.0
4,inv_005,c_01,cls_01,stu_005,2025-02-01,150.0


In [35]:
credit_notes.head()

,id,centre_id,class_id,student_id,credit_note_date,total_amount
0,cr_001,c_01,cls_01,stu_001,2025-05-15,100.0
1,cr_002,c_02,cls_02,stu_002,2025-03-10,50.0
2,cr_003,c_02,cls_03,stu_003,2024-12-01,300.0
3,cr_004,c_01,cls_01,stu_004,2025-01-20,120.0
4,cr_005,c_03,cls_01,stu_005,2025-06-01,200.0


In [36]:
payments.head()

,id,document_id,document_type,amount_paid,payment_date
0,pay_001,inv_001,invoice,150.0,2025-05-10
1,pay_002,cr_003,credit_note,100.0,2025-02-01
2,pay_003,inv_002,invoice,200.0,2025-06-10
3,pay_004,inv_004,invoice,100.0,2025-01-10
4,pay_005,cr_006,credit_note,40.0,2025-03-05


In [37]:
def transform_documents(df, doc_type, date_col):
    df = df.rename(columns={
        "id": "document_id",
        date_col: "document_date"
    })
    df["document_type"] = doc_type
    return df

In [43]:
transformed_invoices = transform_documents(invoices, "invoice", "invoice_date")
transformed_credit_notes = transform_documents(credit_notes, "credit_note", "credit_note_date")
all_documents = pd.concat([transformed_invoices, transformed_credit_notes], ignore_index=True)

In [44]:
transformed_invoices.head()

,document_id,centre_id,class_id,student_id,document_date,total_amount,document_type
0,inv_001,c_01,cls_01,stu_001,2025-05-01,300.0,invoice
1,inv_002,c_01,cls_02,stu_002,2025-06-01,200.0,invoice
2,inv_003,c_02,cls_01,stu_003,2025-01-01,500.0,invoice
3,inv_004,c_03,cls_03,stu_004,2024-12-15,400.0,invoice
4,inv_005,c_01,cls_01,stu_005,2025-02-01,150.0,invoice


In [45]:
transformed_credit_notes.head()

,document_id,centre_id,class_id,student_id,document_date,total_amount,document_type
0,cr_001,c_01,cls_01,stu_001,2025-05-15,100.0,credit_note
1,cr_002,c_02,cls_02,stu_002,2025-03-10,50.0,credit_note
2,cr_003,c_02,cls_03,stu_003,2024-12-01,300.0,credit_note
3,cr_004,c_01,cls_01,stu_004,2025-01-20,120.0,credit_note
4,cr_005,c_03,cls_01,stu_005,2025-06-01,200.0,credit_note


In [63]:
all_documents.head()

,document_id,centre_id,class_id,student_id,document_date,total_amount,document_type
0,inv_001,c_01,cls_01,stu_001,2025-05-01,300.0,invoice
1,inv_002,c_01,cls_02,stu_002,2025-06-01,200.0,invoice
2,inv_003,c_02,cls_01,stu_003,2025-01-01,500.0,invoice
3,inv_004,c_03,cls_03,stu_004,2024-12-15,400.0,invoice
4,inv_005,c_01,cls_01,stu_005,2025-02-01,150.0,invoice


In [48]:
payments_grouped = payments.groupby("document_id")["amount_paid"].sum().reset_index()

In [55]:
documents = all_documents.merge(payments_grouped, on="document_id", how="left").fillna(0.0)

In [58]:
documents["outstanding_amount"] = documents["total_amount"] - documents["amount_paid"]

In [61]:
filtered_documents = documents[documents["outstanding_amount"] > 0]

In [62]:
filtered_documents.head()

,document_id,centre_id,class_id,student_id,document_date,total_amount,document_type,amount_paid,outstanding_amount
0,inv_001,c_01,cls_01,stu_001,2025-05-01,300.0,invoice,150.0,150.0
2,inv_003,c_02,cls_01,stu_003,2025-01-01,500.0,invoice,0.0,500.0
3,inv_004,c_03,cls_03,stu_004,2024-12-15,400.0,invoice,100.0,300.0
4,inv_005,c_01,cls_01,stu_005,2025-02-01,150.0,invoice,50.0,100.0
6,inv_007,c_03,cls_01,stu_007,2025-03-20,300.0,invoice,0.0,300.0


In [ ]:
def assign_bucket(row):
    age = (AS_AT_DATE - row["document_date"]).days
    for bucket, (start, end) in BUCKETS.items():
        if start <= age <= end:
            return bucket
    return None

In [68]:
filtered_documents["ageing_bucket"] = filtered_documents.apply(assign_bucket, axis=1)

/var/folders/nl/8kx351f92mn4rxj66c9k9h640000gp/T/ipykernel_99913/3407079785.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_documents["ageing_bucket"] = filtered_documents.apply(assign_bucket, axis=1)


In [70]:
filtered_documents[["document_id", "outstanding_amount", "ageing_bucket"]].head()

,document_id,outstanding_amount,ageing_bucket
0,inv_001,150.0,day_90
2,inv_003,500.0,day_180_and_above
3,inv_004,300.0,day_180_and_above
4,inv_005,100.0,day_180
6,inv_007,300.0,day_120


In [88]:
buckets = filtered_documents.pivot_table(
    index=['document_id'],
    columns='ageing_bucket',
    values='outstanding_amount',
    fill_value=0.0
).reset_index()

buckets

ageing_bucket,document_id,day_120,day_150,day_180,day_180_and_above,day_30,day_60,day_90
0,cr_001,0.0,0.0,0.0,0.0,0.0,100.0,0.0
1,cr_002,50.0,0.0,0.0,0.0,0.0,0.0,0.0
2,cr_003,0.0,0.0,0.0,200.0,0.0,0.0,0.0
3,cr_004,0.0,0.0,120.0,0.0,0.0,0.0,0.0
4,cr_005,0.0,0.0,0.0,0.0,0.0,200.0,0.0
5,cr_006,0.0,40.0,0.0,0.0,0.0,0.0,0.0
6,cr_007,0.0,0.0,0.0,0.0,0.0,0.0,110.0
7,inv_001,0.0,0.0,0.0,0.0,0.0,0.0,150.0
8,inv_003,0.0,0.0,0.0,500.0,0.0,0.0,0.0
9,inv_004,0.0,0.0,0.0,300.0,0.0,0.0,0.0


In [91]:
filtered_documents.merge(buckets, on="document_id", how="inner")

,document_id,centre_id,class_id,student_id,document_date,total_amount,document_type,amount_paid,outstanding_amount,ageing_bucket,day_120,day_150,day_180,day_180_and_above,day_30,day_60,day_90
0,inv_001,c_01,cls_01,stu_001,2025-05-01,300.0,invoice,150.0,150.0,day_90,0.0,0.0,0.0,0.0,0.0,0.0,150.0
1,inv_003,c_02,cls_01,stu_003,2025-01-01,500.0,invoice,0.0,500.0,day_180_and_above,0.0,0.0,0.0,500.0,0.0,0.0,0.0
2,inv_004,c_03,cls_03,stu_004,2024-12-15,400.0,invoice,100.0,300.0,day_180_and_above,0.0,0.0,0.0,300.0,0.0,0.0,0.0
3,inv_005,c_01,cls_01,stu_005,2025-02-01,150.0,invoice,50.0,100.0,day_180,0.0,0.0,100.0,0.0,0.0,0.0,0.0
4,inv_007,c_03,cls_01,stu_007,2025-03-20,300.0,invoice,0.0,300.0,day_120,300.0,0.0,0.0,0.0,0.0,0.0,0.0
5,inv_008,c_01,cls_02,stu_008,2025-06-20,100.0,invoice,0.0,100.0,day_30,0.0,0.0,0.0,0.0,100.0,0.0,0.0
6,cr_001,c_01,cls_01,stu_001,2025-05-15,100.0,credit_note,0.0,100.0,day_60,0.0,0.0,0.0,0.0,0.0,100.0,0.0
7,cr_002,c_02,cls_02,stu_002,2025-03-10,50.0,credit_note,0.0,50.0,day_120,50.0,0.0,0.0,0.0,0.0,0.0,0.0
8,cr_003,c_02,cls_03,stu_003,2024-12-01,300.0,credit_note,100.0,200.0,day_180_and_above,0.0,0.0,0.0,200.0,0.0,0.0,0.0
9,cr_004,c_01,cls_01,stu_004,2025-01-20,120.0,credit_note,0.0,120.0,day_180,0.0,0.0,120.0,0.0,0.0,0.0,0.0
